In [8]:
from pydicom.filereader import dcmread
import os
import numpy as np
import math
import glob
import re

In [9]:
import pandas as pd

In [10]:
import scipy
import numpy as np
import scipy.io as spio


In [11]:
homedir = '/media/raghuram/My Passport/dicom_seg/TCGA-LGG'
os.chdir(homedir)
folder_dict = {}

In [12]:
# dicom_df = pd.read_csv('dicom_filenames.csv', names=['filename'])


In [ ]:
mat_file_dir = '/home/raghuram/Desktop/radiomics/TEXTURES/'
os.chdir(mat_file_dir)
sequence_dict = dict()

In [18]:
homedir = '/media/raghuram/My Passport/dicom_seg/TCGA-LGG'
os.chdir(homedir)

t1_df=pd.read_csv('t1_pre_samples.csv')
t1_post_df=pd.read_csv('t1_post_samples.csv')
t2_df=pd.read_csv('t2_pre_samples.csv')
flair_df=pd.read_csv('flair_samples.csv')

In [19]:
list(t1_post_df['Sequence Name'].values)

['AXIAL T1 POST GAD FATSAT',
 'BRAIN_CONTRAST/T1_AXIAL',
 'T1 AX SE FS POST',
 'AX T1 POST GD FLAIR',
 'POST AX T1W_IR_TSE',
 'AXIAL T1 FSE POST GAD',
 'POST AX T1 FLAIR',
 'MRHR T1 AX POST GAD',
 'T1 AXIAL GD',
 'AX T1 MP SPGR + GAD',
 'POST AX T1 SE',
 'AX T1 MP SPGR+C',
 'POST AX T1 MP SPGR',
 'VOLUMETRIC AXIAL GD',
 'POST AX 3D SPGR',
 'AX 3D SPGR+C',
 '+C AX T1 MP SPGR',
 'C+AX 3D SPGR',
 'AX T1+C',
 'AX 3D SPGR + C',
 'AX T1 SE+C',
 'RA ROUTINE T1 AX +C',
 '+C 3D AXIAL IRSPGR FAST',
 'AX T1 MP SPGR +C',
 '+C AX 3D SPGR',
 'BRAIN_CONTRAST/T1_AXIAL_C+']

In [20]:
sequence_mapping = {'T1CE':list(t1_post_df['Sequence Name'].values), 
                    'T1W':list(t1_df['Sequence Name'].values),
                    'T2F':list(flair_df['Sequence Name'].values),
                    'T2W':list(t2_df['Sequence Name'].values)}

In [21]:
sequence_mapping

{'T1CE': ['AXIAL T1 POST GAD FATSAT',
  'BRAIN_CONTRAST/T1_AXIAL',
  'T1 AX SE FS POST',
  'AX T1 POST GD FLAIR',
  'POST AX T1W_IR_TSE',
  'AXIAL T1 FSE POST GAD',
  'POST AX T1 FLAIR',
  'MRHR T1 AX POST GAD',
  'T1 AXIAL GD',
  'AX T1 MP SPGR + GAD',
  'POST AX T1 SE',
  'AX T1 MP SPGR+C',
  'POST AX T1 MP SPGR',
  'VOLUMETRIC AXIAL GD',
  'POST AX 3D SPGR',
  'AX 3D SPGR+C',
  '+C AX T1 MP SPGR',
  'C+AX 3D SPGR',
  'AX T1+C',
  'AX 3D SPGR + C',
  'AX T1 SE+C',
  'RA ROUTINE T1 AX +C',
  '+C 3D AXIAL IRSPGR FAST',
  'AX T1 MP SPGR +C',
  '+C AX 3D SPGR',
  'BRAIN_CONTRAST/T1_AXIAL_C+'],
 'T1W': ['AXIAL T1',
  'T1 AX  SE',
  'BRAIN_STRYKER/T1_AXIAL',
  'BRAIN_STRYKER/T1_AX',
  'T1 AX SE',
  'AX T1 PRE GD',
  'AX T1 2.5MM FOR SURGERY',
  'RA ROUTINE T1 AX',
  'AX T1_IR_TSE_5MM',
  'AXIAL T1 FSE',
  'AX T1',
  'T1 AX',
  'TIR T1 AX',
  'AX T1 MP SPGR',
  'AX T1 MP SPGR PRE GAD',
  'AX T1 SE',
  'PRE AX T1 MP SPGR',
  'TI AX SE',
  '3D AXIAL IRSPGR FAST',
  'MP RAGE AXIAL',
  'AX 3D S

In [ ]:
def categorize_mat_files():
    mat_file_list = glob.glob(os.path.join(mat_file_dir, '*.mat'))
    for idx, mat_file in enumerate(mat_file_list):
        sequence_name = mat_file.split('_')[1].split('.mat')[0]
        patient_name = mat_file.split('_')[0].split('/')[-1]
        print(patient_name, sequence_name)

In [ ]:
def fetch_magnetization_strength(dicom_file):
    try:
        mag_strength = dicom_file.MagneticFieldStrength
        if mag_strength>1000:
            return mag_strength/10000
        return mag_strength
    except Exception:
        raise Exception

In [ ]:
def fetch_scanner_model(dicom_file):
    try:
        return dicom_file.ManufacturerModelName
    except Exception:
        raise Exception

In [ ]:
def fetch_scanner_manufacturer(dicom_file):
    try:
        return dicom_file.Manufacturer
    except Exception:
        raise Exception

In [ ]:
def fetch_repetition_time(dicom_file):
    try:
        return dicom_file.RepetitionTime
    except Exception:
        raise Exception
    

In [ ]:
def fetch_excitation_time(dicom_file):
    try:
        return dicom_file.EchoTime
    except Exception:
        raise Exception

In [ ]:
def fetch_scanning_seq_mri(dicom_file):
    try:
        return dicom_file.SeriesDescription
    
    except Exception:
        raise Exception

In [ ]:
def fetch_body_part_examined(dicom_file):
    try:
        return dicom_file.BodyPartExamined
    except Exception as e:
        raise Exception

In [ ]:
row_list = []

In [ ]:
for idx, row in dicom_df.iterrows():
    
    try:
#         dict_to_append = {'mag_field_strength':'', 'scanner_manufacturer':'', 'scanner_model':'', 'repetition_time':'', 'excitation_time':'',
#                   'scanning_seq_mri':'', 'filename':'', 'body_part_examined':''}
        dicom_data = dcmread(row[0])
        print(dicom_data)
        break
#         dict_to_append['filename'] = row[0]
#         dict_to_append['excitation_time'] = fetch_excitation_time(dicom_data)
#         dict_to_append['repetition_time'] = fetch_repetition_time(dicom_data)
#         dict_to_append['mag_field_strength'] = fetch_magnetization_strength(dicom_data)
#         dict_to_append['scanner_manufacturer'] = fetch_scanner_manufacturer(dicom_data)
#         dict_to_append['scanner_model'] = fetch_scanner_model(dicom_data)
#         dict_to_append['scanning_seq_mri'] = dicom_data.SeriesDescription
#         dict_to_append['body_part_examined'] = fetch_body_part_examined(dicom_data)
#         row_list.append(dict_to_append)
    
    except Exception as e:
        print('Exception : {} has occurred for file at position {}'.format(e, idx))
    

In [ ]:
import csv

In [ ]:
keys = row_list[0].keys()
with open(os.path.join(os.getcwd(),'dicom_file_params.csv'), 'w') as outfile:
    dict_writer = csv.DictWriter(outfile, keys)
    dict_writer.writeheader()
    dict_writer.writerows(row_list)

In [34]:
dicom_file_params_df = pd.read_csv('dicom_file_params.csv')
dicom_file_params_df['scanning_seq_mri'] = dicom_file_params_df['scanning_seq_mri'].str.upper()
dicom_file_params_df = dicom_file_params_df[dicom_file_params_df['scanner_manufacturer'] != 'Hitachi Medical Corporation']
dicom_file_params_df = dicom_file_params_df[(dicom_file_params_df['scanning_seq_mri'] != 'BRAIN_PITUITARY/PD_T2_AXIA') &
                    (dicom_file_params_df['scanning_seq_mri'] != 'BRAIN_CONTRAST/DOUBLE_ECHO') 
                                            & (dicom_file_params_df['scanning_seq_mri'] != 'VOLUMETRIC AXIAL')
                                          & (dicom_file_params_df['scanning_seq_mri'] != 'AX') ]
# dicom_file_params_df.to_csv('dicom_file_params.csv')
#Confirmation done for drop

In [35]:
standard_mri_seq_name = []

In [36]:
for idx, row in dicom_file_params_df.iterrows():
#     print(row['scanning_seq_mri'])
    if row['scanning_seq_mri'] in sequence_mapping['T1CE']:
        standard_mri_seq_name.append('T1CE')
    elif row['scanning_seq_mri'] in sequence_mapping['T1W']:
        standard_mri_seq_name.append('T1W')
    elif row['scanning_seq_mri'] in sequence_mapping['T2W']:
        standard_mri_seq_name.append('T2W')
    elif row['scanning_seq_mri'] in sequence_mapping['T2F']:
        standard_mri_seq_name.append('T2F')
    else:
        print('Not present {} :{}'.format(idx, row['scanning_seq_mri']))

#     if row['scanning_seq_mri'] 
#     scanning_seq_value = row['scanning_seq_mri'] 
#     for key, value in sequence_mapping.items():
#         if scanning_seq_value in value:
#             standard_mri_seq_name.append(key)
#         else:
#             print(scanning_seq_value, idx)

In [37]:
dicom_file_params_df.shape

(24398, 12)

In [38]:
len(standard_mri_seq_name)

24398

In [ ]:
output = dicom_file_params_df[dicom_file_params_df['scanning_seq_mri'].str.contains('C+')]

In [ ]:
output['scanning_seq_mri'].unique()

In [2]:
t1_pre = ['AXIAL T1', 'T1 AX  SE', 'BRAIN_STRYKER/T1_AXIAL', 'BRAIN_STRYKER/T1_AX',
          'T1 AX SE', 'AX T1 PRE GD', 'AX T1 2.5MM FOR SURGERY', 'RA ROUTINE T1 AX', 'AX T1_IR_TSE_5MM','AXIAL T1 FSE', 
           'AX T1', 'T1 AX', 'TIR T1 AX', 'AX T1 MP SPGR', 'AX T1 MP SPGR PRE GAD', 'AX T1 SE','PRE AX T1 MP SPGR',
         'TI AX SE', '3D AXIAL IRSPGR FAST', 'MP RAGE AXIAL', 'AX 3D SPGR', 'MPRAGE AX 3D SPGR FAST', 'REPEAT AX 3D SPGR' ]

In [1]:
t1_post = ['AXIAL T1 POST GAD FATSAT', 'BRAIN_CONTRAST/T1_AXIAL',  
          'T1 AX SE FS POST', 'AX T1 POST GD FLAIR', 'POST AX T1W_IR_TSE',
          'AXIAL T1 FSE POST GAD', 'POST AX T1 FLAIR', 'MRHR T1 AX POST GAD', 'T1 AXIAL GD', 
           'AX T1 MP SPGR + GAD', 'POST AX T1 SE', 'AX T1 MP SPGR+C', 
           'POST AX T1 MP SPGR', 'VOLUMETRIC AXIAL GD',
           'T1 AXIAL GD', 'AX T1 POST GD FLAIR','POST AX 3D SPGR', 'AX 3D SPGR+C', '+C AX T1 MP SPGR','C+AX 3D SPGR', 'AX T1+C',
       'AX 3D SPGR + C', 'AX T1 MP SPGR+C', 'AX T1 SE+C', 'RA ROUTINE T1 AX +C', '+C 3D AXIAL IRSPGR FAST', 'AX T1 MP SPGR +C', '+C AX 3D SPGR', 'C+AX 3D SPGR', 'BRAIN_CONTRAST/T1_AXIAL_C+' ]

In [3]:
t2 = ['AXIAL T2', 'BRAIN_STRYKER/T2_AX_DOUBLE', 'RETRO_ORBIT/T2_LATE_ECHO_B','BRAIN_PITUITARY/PD_T2_AXIA',
      'T2W_TSE', 'AXIAL  T2', 'AX T2 FR-FSE RF2 150', 'RA ROUTINE T2 AX','AX T2 FSE', 'AX T2W_DRIVE', 
      'AXIAL T2 FSE HI-RES', 'AX T2','MRHG T2 AXIALS', 'T2 AXIALS', 'MRHR T2 AX','T2 AX', 'TSE T2 AXIALS', 
      'AX T2 FSE INTER', '(OPTIONAL) AX FSE T2 INTER ', 'PROP T2 TRF','(OPTIONAL)AX T2 FSE INTER', 'AX T2 FSE H', 
      'AX FSE T2 INTER',' T2PROP', '(OPT) AX T2 FSE INTER', 'T2PROP', '(OPT)AX T2 FSE INTER', 'AX T2 FSE  FOR WAND',
     'DUAL AX SE', 'AXIAL FSE','BRAIN_STRYKER/T2_AX_DOUBLE', 'RETRO_ORBIT/T2_LATE_ECHO_B', 'T2W_TSE', 'AXIAL  T2',
     'AX T2 FR-FSE RF2 150']

In [4]:
flair = ['AXIAL FLAIR', 'T2 AX FLAIR', 'BRAIN_STRYKER/FLAIR',
       'BRAIN_CONTRAST/FLAIR', 'AX T1 POST GD FLAIR',
       'AX FLAIR 2.5MM FOR SURGERY', 'RA ROUTINE FLAIR AX',
       'AXIAL FLAIR RF2  150', 'FLAIR_LONGTR', 'AX T2 FLAIR',
       'POST AX T1 FLAIR', 'FLAIR AXIALS', 'MRHR FLAIR AX', 'FLAIR AX',
       'AX FLAIR IRFSE H', 'AXIAL T2 FLAIR']

In [5]:
t1_post_dict = {}
t1_pre_dict = {}
t2_pre_dict = {}
flair_dict = {}

# Need to check if AX T1 POST GD FLAIR is T1 or FLAIR?

In [14]:
def write_to_csv(input_list, input_dict, filename):
    for submodality in input_list:
    #     print('Submodality is {} '.format(submodality))
        value = dicom_file_params_df[dicom_file_params_df['scanning_seq_mri'] == submodality].shape[0]
        input_dict[submodality] = value
    pd.DataFrame(list(input_dict.items()), columns=['Sequence Name', 'Values']).to_csv(filename+'.csv')

In [15]:
write_to_csv(t1_post, t1_post_dict, 't1_post_samples')
write_to_csv(t1_pre, t1_pre_dict, 't1_pre_samples')
write_to_csv(t2, t2_pre_dict, 't2_pre_samples')
write_to_csv(flair, flair_dict, 'flair_samples')